In [1]:
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
from matplotlib.backends.backend_pdf import PdfPages


### **DCE组学特征提取前确认**

In [6]:
# base_dir = '/app/liucd/CODE/zunyi/data/data_299+33'
# base_dir = '/app/liucd/deeplearn_dec/data/syf/old/'
base_dir = '/app/liucd/CODE/sd/data/data_dec/'
# base_dir = '/app/liucd/CODE/xian/data/data_dec/'

image_dir = 'ADC'
imagereg_dir = 'ADC_Reg'
label_dir = 'DWI_label/'

# image_dir = 'DCE'
# imagereg_dir = 'DCE_Reg'
# label_dir = 'DCE_label/'

image_paths = sorted(glob.glob(os.path.join(base_dir, image_dir, '*.nii.gz')))
imagereg_paths = sorted(glob.glob(os.path.join(base_dir, imagereg_dir, '*.nii.gz')))
label_paths = sorted(glob.glob(os.path.join(base_dir, label_dir, '*.nii.gz')))

In [7]:
image_paths[:4], 
imagereg_paths[:4], label_paths[:4]

(['/app/liucd/deeplearn_dec/data/syf/old/ADC_Reg/2012_05_30_2607877_ADC2_Reg_0000.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/ADC_Reg/2012_07_17_2646519_ADC2_Reg_0000.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/ADC_Reg/2013_03_27_2833724_ADC2_Reg_0000.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/ADC_Reg/2013_03_28_2839017_ADC2_Reg_0000.nii.gz'],
 ['/app/liucd/deeplearn_dec/data/syf/old/DWI_label/2012_05_30_2607877_DWI.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/DWI_label/2012_07_17_2646519_DWI.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/DWI_label/2013_03_27_2833724_DWI.nii.gz',
  '/app/liucd/deeplearn_dec/data/syf/old/DWI_label/2013_03_28_2839017_DWI.nii.gz'])

In [4]:
def Contour(label_path):
    import SimpleITK as sitk
    
    label_image = sitk.ReadImage(label_path, sitk.sitkInt32)
    dilate_image = sitk.BinaryDilate(label_image, (2, 2, 0))
    label_arr = sitk.GetArrayFromImage(label_image)
    dilate_arr = sitk.GetArrayFromImage(dilate_image)
    contour_arr = dilate_arr - label_arr
    contour_image = sitk.GetImageFromArray(contour_arr)
    contour_image.SetDirection(label_image.GetDirection())
    contour_image.SetOrigin(label_image.GetOrigin())
    contour_image.SetSpacing(label_image.GetSpacing())
    
    return contour_image, contour_arr



In [6]:
def plotshow(image_arr, imagereg_arr, label_arr):
    
    
    label_sum =label_arr.sum(axis=1).sum(axis=1) 
    select_idxs = np.nonzero(label_sum)[0]
    len_idx = len(select_idxs)
    select_idxs = select_idxs[len_idx//2 - 4 : len_idx//2 + 6: 2] if len_idx >=7 else select_idxs
    print(select_idxs)
    
    figure = plt.figure()
    for i, select_idx in enumerate(select_idxs):
        raw_arr = np.clip(image_arr[select_idx], a_min=0, a_max=2000)
        reg_arr = np.clip(imagereg_arr[select_idx], a_min=0, a_max=2000)
        
        scaled_arr = (raw_arr - raw_arr.min()) / raw_arr.max() * 255
        scaled_regarr = (reg_arr - reg_arr.min()) / reg_arr.max() * 255
        
        rgb_img_arr = np.array([scaled_arr, scaled_arr, scaled_arr])
        rgb_reg_arr = np.array([scaled_regarr, scaled_regarr, scaled_regarr])
        
        rgb_lab_arr = np.array([label_arr[select_idx] * 255, np.zeros_like(label_arr[select_idx]), np.zeros_like(label_arr[select_idx])])

        merge_arr = rgb_img_arr + rgb_lab_arr
        merge_regarr = rgb_reg_arr + rgb_lab_arr
        plt.subplot(1, 2, 1)
        plt.imshow(np.clip(merge_arr, 0, 255).astype(np.int16).transpose(1, 2, 0), 'gray')
        plt.subplot(1, 2, 2)
        plt.imshow(np.clip(merge_regarr, 0, 255).astype(np.int16).transpose(1, 2, 0), 'gray')
        
    # plt.show()
        
    return figure

In [8]:
# DCE
with PdfPages('syf_stage2_dec_adc.pdf') as pdf:
    for image_path, imagereg_path, label_path in zip(image_paths, imagereg_paths, label_paths):
        
            
        print(image_path.split('/')[-1], label_path.split('/')[-1], )
        imagereg_arr = sitk.GetArrayFromImage(sitk.ReadImage(imagereg_path))

        image_arr = sitk.GetArrayFromImage(sitk.ReadImage(image_path))
        _, counter_arr = Contour(label_path)
        label_arr = counter_arr #  赋值给自定义函数里的变量
        
        label_sum =label_arr.sum(axis=1).sum(axis=1) 
        select_idxs = np.nonzero(label_sum)[0]
        len_idx = len(select_idxs)
        select_idxs = select_idxs[len_idx//2 - 2 : len_idx//2 + 3: 1] if len_idx >=5 else select_idxs
        print(select_idxs)

        
        for i, select_idx in enumerate(select_idxs):
            figure = plt.figure()
            raw_arr = np.clip(image_arr[select_idx], a_min=0, a_max=2000)
            reg_arr = np.clip(imagereg_arr[select_idx], a_min=0, a_max=2000)

            scaled_arr = (raw_arr - raw_arr.min()) / raw_arr.max() * 255
            scaled_regarr = (reg_arr - reg_arr.min()) / reg_arr.max() * 255

            rgb_img_arr = np.array([scaled_arr, scaled_arr, scaled_arr])
            rgb_reg_arr = np.array([scaled_regarr, scaled_regarr, scaled_regarr])

            rgb_lab_arr = np.array([label_arr[select_idx] * 255, np.zeros_like(label_arr[select_idx]), np.zeros_like(label_arr[select_idx])])

            merge_arr = rgb_img_arr + rgb_lab_arr
            merge_regarr = rgb_reg_arr + rgb_lab_arr
            plt.subplot(1, 2, 1)
            plt.imshow(np.clip(merge_arr, 0, 255).astype(np.int16).transpose(1, 2, 0), 'gray')
            plt.text(0, 0, label_path.split('/')[-1], color='r', fontsize=10)
            
            plt.subplot(1, 2, 2)
            plt.imshow(np.clip(merge_regarr, 0, 255).astype(np.int16).transpose(1, 2, 0), 'gray')
            
            # plt.show()
            
            pdf.savefig(figure)
            
        

2012_05_30_2607877_ADC_0000.nii.gz 2012_05_30_2607877_DWI.nii.gz
[20 21 22 23]
2012_07_17_2646519_ADC_0000.nii.gz 2012_07_17_2646519_DWI.nii.gz
[20 21 22 23 24]
2013_03_27_2833724_ADC_0000.nii.gz 2013_03_27_2833724_DWI.nii.gz
[10 11 12 13 14]
2013_03_28_2839017_ADC_0000.nii.gz 2013_03_28_2839017_DWI.nii.gz
[13 14 15 16 17]
2013_07_01_2910046_ADC_0000.nii.gz 2013_07_01_2910046_DWI.nii.gz
[1 2 3 4 5]


/tmp/ipykernel_3319858/2308072110.py:26: RuntimeWarning: invalid value encountered in divide
  scaled_regarr = (reg_arr - reg_arr.min()) / reg_arr.max() * 255
/tmp/ipykernel_3319858/2308072110.py:40: RuntimeWarning: invalid value encountered in cast
  plt.imshow(np.clip(merge_regarr, 0, 255).astype(np.int16).transpose(1, 2, 0), 'gray')
/tmp/ipykernel_3319858/2308072110.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure = plt.figure()


2013_10_30_3009763_ADC_0000.nii.gz 2013_10_30_3009763_DWI.nii.gz
[15 16 17 18 19]
2013_12_09_1852564_ADC_0000.nii.gz 2013_12_09_1852564_DWI.nii.gz
[ 7  8  9 10 11]
2013_12_30_973491_ADC_0000.nii.gz 2013_12_30_973491_DWI.nii.gz
[ 7  8  9 10 11]
2014_03_13_8024103_ADC_0000.nii.gz 2014_03_13_8024103_DWI.nii.gz
[4 5 6]
2014_03_18_3106649_ADC_0000.nii.gz 2014_03_18_3106649_DWI.nii.gz
[12 13 14 15 16]
2014_03_27_3117884_ADC_0000.nii.gz 2014_03_27_3117884_DWI.nii.gz
[13 14 15]
2014_04_08_2538259_ADC_0000.nii.gz 2014_04_08_2538259_DWI.nii.gz
[17 18 19 20 21]
2014_04_09_3122269_ADC_0000.nii.gz 2014_04_09_3122269_DWI.nii.gz
[16 17 18 19]
2014_04_11_3125784_ADC_0000.nii.gz 2014_04_11_3125784_DWI.nii.gz
[14 15 16 17 18]
2014_04_14_3128987_ADC_0000.nii.gz 2014_04_14_3128987_DWI.nii.gz
[20 21 22 23 24]
2014_05_13_3156436_ADC_0000.nii.gz 2014_05_13_3156436_DWI.nii.gz
[13 14 15 16 17]
2014_05_29_1788908_ADC_0000.nii.gz 2014_05_29_1788908_DWI.nii.gz
[14 15 16 17 18]
2014_06_04_3176054_ADC_0000.nii.gz 2